In [48]:
using Distributions, LinearAlgebra, SpecialFunctions

In [49]:
function mvloggamma(a :: AbstractFloat, d :: Int)
    return d*(d-1)/4 * log(pi) + sum(log.(gamma.(a .+ (Vector(1:d) .- 1) ./ 2)))
end

function mvdigamma(a :: AbstractFloat, d :: Int)
    return sum(digamma.(a .+ (Vector(1:d) .- 1) ./ 2))
end;

In [54]:
p = 3
v = 3
L = tril(rand(3, 3))
P = L * L'
dist = InverseWishart(v, P)

InverseWishart{Float64, PDMats.PDMat{Float64, Matrix{Float64}}}(
df: 3.0
Ψ: [0.3644626975360783 0.38138056271346493 0.556480922755821; 0.38138056271346493 0.4450102536718745 0.7073787197523644; 0.556480922755821 0.7073787197523644 1.2011861478779111]
)


In [ ]:
function loglikinvwishart(v, P, X)
    dim = size(P, 1)
    return v/2 * log(det(P)) - 1/2 * tr(P * inv(X)) - v*dim/2 * log(2) - mvloggamma(v/2, dim) - (v + dim + 1)/2 * log(det(X))
end

loglikinvwishart (generic function with 1 method)

In [59]:
sampleL = tril(rand(3, 3))
sample = sampleL * sampleL'
loglikelihood(dist, sample)

-12.417251348755256

In [60]:
loglikinvwishart(v, P, sample)

-12.1295692763035

In [66]:
log(det(P)^(v/2)/(2^(v*p/2) * exp(mvloggamma(v/2, p))) * det(sample)^(-(v + p + 1)/2) * exp(- 1/2 * tr(P * inv(sample))))

-12.1295692763035

In [78]:
delta = 1e-8
grad = Matrix{Float64}(undef, 3, 3)
new = copy(P)
for i in 1:3
    for j in 1:3
        println(i, j)
        new[i, j] += delta
        # new[j, i] += delta
        # new_dist = InverseWishart(v, new)
        # grad[i, j] = (loglikelihood(new_dist, sample) - loglikelihood(dist, sample))/delta
        grad[i, j] = (loglikinvwishart(v, new, sample) - loglikinvwishart(v, P, sample))/delta
        new[i, j] -= delta
        # new[j, i] -= delta
    end
end

grad

11
12
13
21
22
23
31
32
33


3×3 Matrix{Float64}:
  278.156  -526.896   181.349
 -526.896  1047.9    -372.686
  181.349  -372.686   135.534

In [79]:
v/2 .* inv(P) - 1/2 .* inv(sample)

3×3 Matrix{Float64}:
  278.156  -526.895   181.349
 -526.895  1047.9    -372.686
  181.349  -372.686   135.534

In [80]:
(loglikinvwishart(v + delta, P, sample) - loglikinvwishart(v - delta, P, sample))/(2 * delta)

-4.910278672554114

In [81]:
dist_delta_1 = InverseWishart(v + delta, P)
dist_delta_2 = InverseWishart(v - delta, P)
(loglikelihood(dist_delta_1, sample) - loglikelihood(dist_delta_2, sample))/(2 * delta)

-3.076945542801468

In [82]:
1/2 * log(det(P)) - p/2 * log(2) - mvdigamma(v/2, p) - 1/2 * log(det(sample))

-5.491494245029011

In [26]:
Vector(1:3) .- 1

3-element Vector{Int64}:
 0
 1
 2

In [34]:
#x = Matrix{Float64}(I, 2, 2) .* 4
x = copy(P)
LAPACK.potrf!('L', x)

([0.5751229707796237 0.07874565388829678 0.46941146165009473; 0.13691968133623833 0.668135299731813 0.27601695570515317; 0.816193206495945 0.24585445796511693 0.1743271304178911], 0)

In [35]:
LAPACK.potri!('L', x)

3×3 Matrix{Float64}:
  61.4936    0.0787457   0.469411
  15.5896    6.69563     0.276017
 -43.8158  -12.1083     32.9056

In [39]:
inv(P)

3×3 Matrix{Float64}:
  61.4936   15.5896   -43.8158
  15.5896    6.69563  -12.1083
 -43.8158  -12.1083    32.9056

In [42]:
x = [1 2 3; 4 5 6; 7 8 9]
x_vech = view(x, [3 * (j - 1) + i for j in 1:3 for i in j:3])

6-element view(::Vector{Int64}, [1, 2, 3, 5, 6, 9]) with eltype Int64:
 1
 4
 7
 5
 8
 9

In [43]:
x

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [44]:
x_vech[1] = 0
x

3×3 Matrix{Int64}:
 0  2  3
 4  5  6
 7  8  9